# Description du Notebook

Ce notebook permet d'entraîner un modèle de NER en français avec spaC en se basant les infos de voyage. 

Les étapes incluent : 
- l'importation, le nettoyage et la préparation des données annotées,
- l'entraînement du modèle. 

Ensuite, on fait un comparaison de performances entre le modèle personnalisé et le modèle spaCy préentraîné, avec des visualisations pour évaluer la diversité et le nombre d'entités détectées par chaque modèle.


In [6]:
import spacy
from spacy.tokens import DocBin
import pandas as pd
import ast
from random import shuffle
import subprocess
import time
from tqdm import tqdm

# Liste des modèles spaCy à tester
models = {
    "sm": "fr_core_news_sm",
    "md": "fr_core_news_md",
    "lg": "fr_core_news_lg"
}

# Chargement des modèles spaCy
nlp_models = {}
for model_key, model_name in models.items():
    print(f"Chargement du modèle spaCy {model_key}...")
    nlp_models[model_key] = spacy.load(model_name)
    print(f"Modèle spaCy {model_key} chargé avec succès")

# Fonction de nettoyage du texte
def clean_text(text):
    return text.replace('\r', '').strip()

# Fonction pour vérifier l'alignement des entités
def verify_entity_alignment(text, entities):
    verified_entities = []
    for ent in entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        if text[start:end] == text[start:end]:  # Vérification basique d'alignement
            verified_entities.append(ent)
        else:
            print(f"Skipping entity: {text[start:end]} in text: {text}")
    return verified_entities

# Fonction pour préparer les données au format spaCy
def prepare_data_for_spacy_from_csv(df):
    # Vérification de l'existence de la colonne 'spacy_ner_tags'
    if 'spacy_ner_tags' not in df.columns:
        print("La colonne 'spacy_ner_tags' n'existe pas dans le DataFrame.")
        # Créer une liste vide de la même longueur que le DataFrame
        df['spacy_ner_tags'] = [[] for _ in range(len(df))]  # Liste vide pour chaque ligne
    else:
        print("La colonne 'spacy_ner_tags' est présente.")
    
    return df


# Charger les fichiers CSV
def load_dataset(file_path):
    print(f"Chargement du dataset à partir de {file_path}...")
    df = pd.read_csv(file_path, sep=';', encoding='utf-8', on_bad_lines='skip')
    print("Colonnes du dataset:", df.columns)
    return df

# Chargement des fichiers CSV
df_text = load_dataset("C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/raw/generated_dataset/text.csv")
df_token = load_dataset("C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/raw/generated_dataset/token.csv")

# Vérification et nettoyage des colonnes de texte
if 'text' in df_text.columns:
    df_text['text'] = df_text['text'].apply(clean_text)
else:
    print("La colonne 'text' n'existe pas dans df_text")

if 'text' in df_token.columns:
    df_token['text'] = df_token['text'].apply(clean_text)
else:
    print("La colonne 'text' n'existe pas dans df_token")

# Afficher les premières lignes pour vérifier
print(df_text.head())
print(df_token.head())

# Préparer les données pour spaCy
df_text = prepare_data_for_spacy_from_csv(df_text)
df_token = prepare_data_for_spacy_from_csv(df_token)

# Sauvegarder les données converties au format spaCy
def save_to_spacy_format(df, output_path):
    doc_bin = DocBin()  # Utilisé pour sauvegarder les données spaCy
    for text, annotations in zip(df['text'], df['spacy_ner_tags']):
        doc = nlp_models["sm"](text)  # Utilisation du modèle de base pour tokeniser
        ents = verify_entity_alignment(doc.text, annotations)
        spans = create_entity_spans(doc, ents)
        doc.ents = spans
        doc_bin.add(doc)
    doc_bin.to_disk(output_path)

# Sauvegarder les données converties pour chaque dataset
save_to_spacy_format(df_text, "C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/processed/processed_training_data_text.spacy")
save_to_spacy_format(df_token, "C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/processed/processed_training_data_token.spacy")
print("Les données ont été sauvegardées au format spaCy.")

# Fonction pour créer les spans d'entités dans spaCy
def create_entity_spans(doc, verified_entities):
    ents = []
    for ent in verified_entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span:
            ents.append(span)
    return ents

# Fonction pour entraîner le modèle
def train_model(config_path, output_path, training_data_path):
    print("Génération de la configuration...")
    subprocess.run(f"python -m spacy init config {config_path} --lang fr --pipeline ner --optimize efficiency --force", shell=True)
    print("Configuration générée.")

    print("Début de l'entraînement du modèle...")
    process = subprocess.Popen(f"python -m spacy train {config_path} --output {output_path} --paths.train {training_data_path} --paths.dev {training_data_path}", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    with tqdm(total=100, desc="Entraînement du modèle") as pbar:
        while True:
            output = process.stdout.readline()
            if output == b"" and process.poll() is not None:
                break
            if output:
                print(output.decode().strip()) 
            pbar.update(1)
            time.sleep(1)

# Entraînement des modèles pour chaque dataset (text et token)
datasets = [
    ("text", "C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/processed/processed_training_data_text.spacy"),
    ("token", "C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/processed/processed_training_data_token.spacy")
]

for dataset_name, dataset_path in datasets:
    for model_key in models.keys():
        print(f"Entraînement pour le modèle {model_key} avec le dataset {dataset_name}...")
        train_model('./configuration.cfg', f'C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/models/spacy/{model_key}', dataset_path)
        print(f"Entraînement du modèle {model_key} terminé pour le dataset {dataset_name}.")

Chargement du modèle spaCy sm...
Modèle spaCy sm chargé avec succès
Chargement du modèle spaCy md...
Modèle spaCy md chargé avec succès
Chargement du modèle spaCy lg...
Modèle spaCy lg chargé avec succès
Chargement du dataset à partir de C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/raw/generated_dataset/text.csv...
Colonnes du dataset: Index(['text', 'CORRECT', 'NOT_FRENCH', 'NOT_TRIP', 'UNKNOWN'], dtype='object')
Chargement du dataset à partir de C:/Users/vikne/Documents/Master 2/Semestre 9/Intelligence artificielle/Travel-Order-Resolver/ai/nlp/dataset/raw/generated_dataset/token.csv...
Colonnes du dataset: Index(['text', 'tokens', 'ner_tags', 'spacy_ner_tags'], dtype='object')
                                                text  CORRECT  NOT_FRENCH  \
0  The journey from the central square of carnoul...        0           1   
1  J'aimerais me rendre de l'aéroport de vendôme-...        1           0   
2  De l'hopital de

KeyboardInterrupt: 